In [2]:
%load_ext autoreload
%autoreload 2

In [25]:
from src.Preprocess.Preprocessor import Preprocessor
Preprocessor().create_negative_sampled_dataset(input_address="../data/SA_history_clicks_of_98.csv", 
                                               output_address="../data/data1.pkl", 
                                               neg_to_pos_ratio=1)

Reading input file ../data/SA_history_clicks_of_98.csv



File created at ../data/data1.pkl


,queryText,packageName,similar
0,توشمال,com.example.masood.yocheneapp,1
1,rush,com.ketchapp.rush,1
2,شب چله,yalda.ir.kr.si.ma.usn,1
3,عشق,com.roman.jadogareeshgh,1
4,کلابی,ir.mmdali.cluby,1
...,...,...,...
21896379,حروف الفبا,com.developer.fshordehc,-1
21896380,رویال,com.robingames.ArcherDefense,-1
21896381,فرار از جنگل,com.negarmobi9.azdastreskharejj,-1
21896382,روبینو,com.missnargess14.chegolie,-1


In [ ]:
import torch
from src.SpanPooling import *

class RepresentationBuilder(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim=300):
        super(RepresentationBuilder, self).__init__()
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.pooling_module = get_pooling_module("avg")
        
    def forward(self, input_ids, attention_mask):
        out = self.embedding(input_ids)
        print(out)
        out = self.pooling_module(out, attention_mask)
        return out


In [ ]:
import torch

class Siamese(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Siamese, self).__init__()
        self.representation_builder = torch.nn.Sequential(
            RepresentationBuilder(vocab_size, embedding_dim),
            torch.nn.ReLU()
        )
        self.loss = torch.nn.CosineEmbeddingLoss(margin=0)
        
    def forward(self, x1, x2):
        x1_repr = self.representation_builder(x1)
        x2_repr = self.representation_builder(x2)
        print(x1_repr)
        print(x2_repr)
        print(self.loss(x1_repr, x2_repr, target=torch.tensor([-1, 1])))
        

In [ ]:
import torch
from src.DatasetHandler import DatasetHandler

class Trainer:
    def __init__(self, vocab_size, embedding_dim, dataset_handler: DatasetHandler):
        self.siamese = Siamese(vocab_size, embedding_dim)
    
    def train(self):
        self.siamese(torch.tensor([[1], [0, 0, 0, 0]]), torch.tensor([[1, 2, 3, 3], [9, 9, 9, 9]]))
        
    def predict_from_query(self, queryText):
        

In [7]:
from src.DatasetHandler import DatasetHandler
# dataset_handler = DatasetHandler("data/SA_history_clicks_of_98.csv")
dataset_handler = DatasetHandler("../data/small.pkl")

Loading tokenizer...
Reading input file...



Creating dataset...
Done


In [11]:
# dataset_handler.df[:100]
# dataset_handler.dataset
# d = dataset_handler.df.to_dict()

547410

In [ ]:
dd = dataset_handler.df.sample(frac=0.01)
dd.to_pickle("small.pkl")

In [ ]:
trainer = Trainer(vocab_size=10, embedding_dim=5, dataset_handler=dataset_handler)
trainer.train()